# Acquire Live Event Ticketing Data

In [1]:
import pandas as pd
import json
import datetime
import requests
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import config
%matplotlib inline

## Acquire Ticketing Data from Ticketmaster

In [2]:
#DMA and Market numbers
#dma_list = [382,362,264]
mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(market,source='ticketmaster,frontgate',startDate=None,endDate=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'source': source,
               'classificationName': 'theatre',
               'size': '200',
               'marketId': market,
               'startDateTime' : startDate,
               'endDateTime' : endDate,
               'apikey': config.tm_key}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj.get('page').get('totalPages')

#Get TicketMaster data, return a dataframe
def getTicketMasterData(pageNumber,market,source='ticketmaster,frontgate',startDate=None,endDate=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'source': source,
               'marketId': market,
               'classificationName' : 'theatre',
               'size': '200',
               'page': pageNumber,
               'startDateTime' : startDate,
               'endDateTime' : endDate,
               'apikey': config.tm_key}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'TM_id': str(event.get('id',{})),
            'TM_name' : str(event.get('name',{})),
            'TM_artist': list(str(attraction.get('name')) for attraction in event['_embedded'].get('attractions',{})),
            'TM_venue' : list(str(venue.get('name')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_city' : list(str(venue.get('city',{}).get('name')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_state' : list(str(venue.get('state',{}).get('stateCode')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue _lat' : list(str(venue.get('location',{}).get('latitude')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_long' :list(str(venue.get('location',{}).get('longitude')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_date' : str(event.get('dates',{}).get('start',{}).get('dateTime',{})),
            'TM_timezone' : str(event.get('dates',{}).get('timezone')),
            'TM_span_multiple_days' : str(event.get('dates',{}).get('spanMultipleDays')),
            'TM_presale_date_start' : list(str(presale.get('startDateTime',{})) for presale in event.get('sales').get('presales',{})),
            'TM_presale_date_end' : list(str(presale.get('endDateTime',{})) for presale in event.get('sales').get('presales',{})),
            'TM_sale_date_start' : str(event.get('sales',{}).get('public',{}).get('startDateTime')),
            'TM_FV_prices': event.get('priceRanges'),
            'TM_promoter': str(event.get('promoter',{}).get('name')),
            'TM_genre' : event.get('classifications'),
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

#Convert timedate information from UTC to local time
def convert_times(df,times_list,tz_col):
    #Loop through each timezone
    df_list = []
    for tz in df[tz_col].unique():
        #Filter rows by timezone
        mask = (df[tz_col] == tz)
        df_local = df.loc[mask]
        #Loop through each datetime row
        for col in times_list:
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
            df_local[col] = pd.to_datetime(df_local[col],errors='coerce').dt.tz_localize('UTC').dt.tz_convert(tz)
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
        df_list.append(df_local)
    df = pd.concat(df_list, axis=0)
    return df

#Explode out columns with nested information
def explode(df,col,index_col):
    df1 = df[col].apply(pd.Series)
    df1 = df1[0].apply(pd.Series)
    df1.index = df[index_col]
    df = df.merge(df1, how='left', left_on=index_col, right_index=True)
    df = df.drop(col,axis=1)
    return df

In [3]:
#Create list US Ticketmaster market numbers -- Look at TM API docs for market number info.
#mkt_list = [40] # 40 is San Antonio and Austin Market

**Ticketmaster API limits the amount of data that can be pulled per day**
- Initial Data Acquisition is returning:
    - Concert events in market 40 (San Antonio & Austin)
    - Occuring between the dates April 1, 2021 and May 1, 2021

In [4]:
tm_df_list = []
p1startDate = '2018-01-01T00:00:00Z'
p1endDate = '2020-03-31T00:00:00Z'

#Create dictionaries of dma/mkt numbers, and number of pages of data
mkt_dict_p1 = dict()
mkt_dict_p2 =dict()

#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict_p1[mkt] = get_number_of_TM_pages(market = mkt,
                                              startDate = p1startDate,
                                              endDate = p1endDate)
    mkt_dict_p2[mkt] = get_number_of_TM_pages(market = mkt,
                                              startDate = p1endDate)
    
#Get market data
for mkt in mkt_dict_p1.keys():
    for current_page in range(0,mkt_dict_p1[mkt]):
        tm_df_list.append(getTicketMasterData(market = mkt,
                                              pageNumber = current_page,
                                              startDate = p1startDate,
                                              endDate = p1endDate))
for mkt in mkt_dict_p2.keys():
    for current_page in range(0,mkt_dict_p2[mkt]):
        tm_df_list.append(getTicketMasterData(market = mkt,
                                              pageNumber = current_page,
                                              startDate = p1endDate))

In [5]:
#Combine all dataframes for full data on events and face_value prices
tm_df = pd.concat(tm_df_list,axis=0)

tm_df.head()

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,TM_span_multiple_days,TM_presale_date_start,TM_presale_date_end,TM_sale_date_start,TM_FV_prices,TM_promoter,TM_genre
0,vvG1HZptU9av1x,Disney Princess: The Concert Brought to you by...,[Disney Princess: The Concert],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-03-06T22:00:00Z,America/Los_Angeles,False,"[2021-05-04T14:00:00Z, 2021-05-06T17:00:00Z, 2...","[2021-05-07T05:00:00Z, 2021-05-07T05:00:00Z, 2...",2021-05-07T17:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
1,vvG1HZpAEWqUaC,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-30T04:00:00Z,America/Los_Angeles,False,[],[],2020-03-09T17:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
2,vvG1HZpAEWQJa7,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-26T03:30:00Z,America/Los_Angeles,False,[],[],2020-03-09T17:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
3,vvG1HZpAEWqkaG,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-29T04:00:00Z,America/Los_Angeles,False,[],[],2020-03-09T17:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
4,vvG1HZpAEWjpaF,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-28T03:30:00Z,America/Los_Angeles,False,[],[],2020-03-09T17:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."


In [6]:
tm_df.shape

(4452, 17)

## Initial Data Preparation

**Events with same name but different dates and markets have the same TM_id. I am dropping duplicates for this initial analysis.**<br>
**Also dropping all parking related ticket sales**

In [7]:
#Drop duplicates
tm_df.drop_duplicates(subset='TM_id',inplace=True)

#Remove parking passes
tm_df = tm_df.loc[~tm_df['TM_venue'].str.lower().str.contains('parking')]
tm_df = tm_df.loc[~tm_df['TM_name'].str.lower().str.contains('parking')]

**Expanding out the price column, genre columnm, and ID column**

In [8]:
#Explode price column
tm_df = explode(tm_df,col='TM_FV_prices',index_col='TM_id')
#Unnest genre info and clean up data in exploded columns
tm_df = explode(tm_df,'TM_genre','TM_id')
for col in ['genre','segment','subGenre']:
    tm_df[col] = tm_df[col].map(lambda x: dict(x).get('name',{}))
tm_df.head()

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,...,max,min,type_x,primary,segment,genre,subGenre,type_y,subType,family
0,vvG1HZptU9av1x,Disney Princess: The Concert Brought to you by...,[Disney Princess: The Concert],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-03-06T22:00:00Z,America/Los_Angeles,...,75.25,35.25,standard,True,Arts & Theatre,Children's Theatre,Children's Theatre,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
1,vvG1HZpAEWqUaC,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-30T04:00:00Z,America/Los_Angeles,...,130.00,50.00,standard,True,Arts & Theatre,Theatre,Musical,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
2,vvG1HZpAEWQJa7,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-26T03:30:00Z,America/Los_Angeles,...,130.00,35.00,standard,True,Arts & Theatre,Theatre,Musical,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
3,vvG1HZpAEWqkaG,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-29T04:00:00Z,America/Los_Angeles,...,130.00,50.00,standard,True,Arts & Theatre,Theatre,Musical,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
4,vvG1HZpAEWjpaF,Anastasia (Touring),[Anastasia (Touring)],Paramount Theatre,Seattle,WA,47.6135664,-122.3319103,2022-01-28T03:30:00Z,America/Los_Angeles,...,130.00,35.00,standard,True,Arts & Theatre,Theatre,Musical,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False


In [9]:
tm_df.shape

(4416, 27)

In [10]:
tm_df.tail(20)

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,...,max,min,type_x,primary,segment,genre,subGenre,type_y,subType,family
75,vvG1VZpI9AEyyX,Dance Now! Miami: Program III featuring Dorian...,[Dance Now!],Amaturo Theater at Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2021-05-16T00:00:00Z,America/New_York,...,40.00,40.00,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
76,vvG1VZpP5AY_rH,Cuban Classical Ballet of Miami Grand Gala,[Cuban Classical Ballet of Miami],The Fillmore Miami Beach at Jackie Gleason The...,Miami Beach,FL,25.792917,-80.13314,2021-06-06T00:00:00Z,America/New_York,...,59.00,39.00,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
77,vvG1VZp_Qc_F5U,Shen Yun,[Shen Yun],Au-Rene Theater at the Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2021-07-18T18:00:00Z,America/New_York,...,200.00,80.00,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
78,vvG1VZpA58w1Z9,South Florida Ballet,[South Florida Ballet Theater],Amaturo Theater at Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2021-11-27T00:00:00Z,America/New_York,...,46.73,23.36,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
79,vvG1VZpI5prnKN,State Ballet Theatre of Ukraine: Cinderella,[],Parker Playhouse,Ft Lauderdale,FL,26.134246,-80.136172,2021-12-30T00:00:00Z,America/New_York,...,65.88,36.16,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
80,vvG1VZ4gwI6581,Alan Chamo: M1ND H4CK3R,[Alan Chamo],Abdo New River Room at the Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2021-11-06T00:00:00Z,America/New_York,...,45.00,35.00,standard,True,Arts & Theatre,Magic & Illusion,Magic,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
81,vvG1VZptrs0ndP,Miss Universe Dress Rehearsal,[Miss Universe],Hard Rock Live,Hollywood,FL,26.0503384,-80.2109577,2021-05-16T17:00:00Z,America/New_York,...,187.00,80.00,standard,True,Arts & Theatre,Fashion,Fashion,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
82,vvG1VZ4gwI6D88,Alan Chamo: M1ND H4CK3R,[Alan Chamo],Abdo New River Room at the Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2021-11-07T00:00:00Z,America/New_York,...,45.00,35.00,standard,True,Arts & Theatre,Magic & Illusion,Magic,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
83,vvG1VZp7QuwnS7,Kevin Nealon,[Kevin Nealon],Amaturo Theater at Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2022-03-13T01:00:00Z,America/New_York,...,45.00,45.00,standard,True,Arts & Theatre,Comedy,Comedy,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
84,vvG1VZpCnmwAPd,Ronald K. Brown: EVIDENCE,[Ronald K Brown],Amaturo Theater at Broward Center,Ft Lauderdale,FL,26.1193,-80.149426,2022-01-07T01:00:00Z,America/New_York,...,59.50,49.50,standard,True,Arts & Theatre,Dance,Dance,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False


In [11]:
#tm_df.to_csv('theatre_tm_events_050121_123122.csv')

In [12]:
#tm_df[tm_df.TM_venue_state == "TX"]